使用 PyMC3
----

如果要用 PyMC2, 可以用如下方式安装：
```
conda install -c pymc pymc
```

如果要用 PyMC3, 可以用如下方式安装：
```
pip install git+https://github.com/pymc-devs/pymc
```

译者注：实际上也完全可以通过 `pip install pymc` 和 `pip install pymc3` 这两种方法来安装 PyMC2 和 PyMC3，我在 macOS High Sierra 上的 Python 3.5.4 和 3.6.3 上面都试过了直接用 pip 来安装，全部成功了。


关于这两个模块的更进一步的详细说明可以参考下面的链接：

* [使用 PyMC2 进行贝叶斯数据分析（Bayesian data analysis）](http://nbviewer.ipython.org/github/isofer/Bayesian-data-analysis-with-PyMC2/blob/master/PyData2013%20boston.ipynb)
* [使用 PyMC3 进行贝叶斯数据分析（Bayesian data analysis）](http://nbviewer.ipython.org/github/twiecki/pymc3_talk/blob/master/bayesian_pymc3.ipynb)

* [黑客（Hackers）的贝叶斯方法（Bayesian-Methods）](http://camdavidsonpilon.github.io/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/)

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from matplotlib import animation
import pandas as pd
import statsmodels.api as sm
import pymc3 as pm

import warnings
warnings.filterwarnings("ignore")

# define invlogit function
def invlogit(x):
    return pm.exp(x) / (1 + pm.exp(x))

/Users/cycleuser/.pyenv/versions/3.5.4/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


### 估计正态分布的均值，Estimating mean of normal distribution


先从一个非常简单的例子开始，给定一个集合，设集合中的元素取样自一个 $\sigma = 1$ 的正态分布中，使用 MCMC 来估计未知参数 $\mu$。

$$
X \sim \mathcal{N}(\mu, 1)
$$

In [5]:
# generate observed data
N = 100
_mu = np.array([10])
y = np.random.normal(_mu, 1, N)

niter = 1000
with pm.Model() as model:
    # define priors
    mu = pm.Flat('mu', shape=_mu.shape)
    
    # define likelihood
    y_obs = pm.Normal('Y_obs', mu=mu, sd=1, observed=y)
    
    # inference
    start = pm.find_MAP()
    step = pm.NUTS()
    trace = pm.sample(niter, step, start, random_seed=123, progressbar=True)

logp = -5,152.5, ||grad|| = 1,001.9: 100%|██████████| 5/5 [00:00<00:00, 1893.93it/s]
100%|██████████| 1500/1500 [00:00<00:00, 2382.14it/s]


In [6]:
plt.hist(trace['mu'][-niter/2:,0], 25, histtype='step');

TypeError: slice indices must be integers or None or have an __index__ method

### 估计正态分布的均值和标准差，Estimating mean and standard deviation of normal distribution

接下来这个例子更进一步，我们这回连分布的参数 $\sigma$ 都不知道了。

$$
X \sim \mathcal{N}(\mu, \sigma^2)
$$

In [ ]:
# generate observed data
N = 100
_mu = np.array([10])
_sigma = np.array([2])
y = np.random.normal(_mu, _sigma, N)

with pm.Model() as model:
    # define priors
    mu = pm.Uniform('mu', lower=0, upper=100, shape=_mu.shape)
    sigma = pm.Uniform('sigma', lower=0, upper=10, shape=_sigma.shape)
    
    # define likelihood
    y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=y)
    
    # inference
    start = pm.find_MAP()
    step = pm.NUTS()
    trace = pm.sample(niter, step, start, random_seed=123, progressbar=True)

In [ ]:
plt.subplot(1,2,1); 
plt.hist(trace['mu'][-niter/2:,0], 25, histtype='step');
plt.subplot(1,2,2); 
plt.hist(trace['sigma'][-niter/2:,0], 25, histtype='step');

### 估计一个线性回归模型的参数，Estimating parameter of a linear regreession model



接下来演示的是使用一个简单的通用线性模型来估计回归参数。其中，设：

$$
y \sim ax + b
$$

以及：

$$
logti(y) \sim ax + b
$$

#### 线性模型（Linear model）

接下来我们可以将线性模型  $$y = ax + b + \epsilon$$ 表示成从下面这个概率分布中进行的一个抽样：

$$
y \sim \mathcal{N}(ax + b, \sigma^2)
$$

或者使用更熟悉的统计矩阵来表述：
$$
y \sim \mathcal{N}(X\beta, \sigma^2)
$$


这时候就可以用 pymc3 来估计参数 $a$, $b$ 和 $\sigma$ 了。

In [ ]:
import os
os.environ['DYLD_FALLBACK_LIBRARY_PATH']='/Users/cliburn/anaconda/lib'

In [ ]:
# observed data
n = 11
a = 6
b = 2
x = np.linspace(0, 1, n)
y = a*x + b + np.random.randn(n)
data = pd.DataFrame(np.array([x, y]).T, columns=['x', 'y'])

with pm.Model() as model:
    # define likelihood
    pm.glm.glm('y ~ x', data, 
               priors={'Intercept': pm.Uniform.dist(-20, 20), 'x': pm.Uniform.dist(-20, 20)})
    
    # inference
    step = pm.NUTS()
    trace = pm.sample(niter, step, random_seed=123, progressbar=True)

In [ ]:
pm.summary(trace)

In [ ]:
pm.traceplot(trace);

In [ ]:
plt.scatter(x, y)
pm.glm.plot_posterior_predictive(trace, samples=50, eval=x)

#### 逻辑回归模型，Logistic model

In [ ]:
# observed data
df = pd.read_csv('https://raw.githubusercontent.com/pymc-devs/pymc/master/pymc/examples/data/HtWt.csv')
df.head(10)

In [ ]:
niter = 1000
with pm.Model() as model:
    # define likelihood
    family = pm.glm.families.Binomial()
    pm.glm.glm('male ~ height + weight', df, family=family) 
    
    # inference
    step = pm.NUTS() 
    trace = pm.sample(niter, step, random_seed=123, progressbar=True)

In [ ]:
df_trace = pm.trace_to_dataframe(trace)
pd.scatter_matrix(df_trace[-1000:], diagonal='kde');

In [ ]:
pm.summary(trace);

In [ ]:
import seaborn as sn
sn.kdeplot(trace['height'], trace['weight'])
matplotlib.rcdefaults()
matplotlib.rcParams.update({'font.size': 10, 'figure.figsize': (6.0, 4.0), 
                            'figure.facecolor': 'white', 'savefig.dpi': 72, 
                            'figure.subplot.bottom': 0.125, 
                            'figure.edgecolor': 'white'})

We can use the logistic regression results to classify subjects as male or female based on their height and weight, using 0.5 as a cutoff, as shown in the plot below. 


接下来我们可以使用逻辑回归的结果，跟根据身高体重来将对象分类，判断是男还是女，使用 0.5 作为临界值（cutoff），如下图表所示：


Green = true positive male,
yellow = true positive female, 
red halo = misclassification. 
Blue line shows the 0.5 cutoff.




In [ ]:
# find posterior mean from last 50% of samples
intercept, height, weight = df_trace[:-niter/2].mean(0)

def predict(w, h, height=height, weight=weight):
    """Predict gender given weight (w) and height (h) values."""
    v = intercept + height*h + weight*w
    return np.exp(v)/(1+np.exp(v))

# calculate predictions on grid
xmin, xmax = df_trace.weight.min(), df_trace.weight.max()
ymin, ymax = df_trace.height.min(), df_trace.height.max()
xs = np.linspace(df.weight.min(), df.weight.max(), 100)
ys = np.linspace(df.height.min(), df.height.max(), 100)
X, Y = np.meshgrid(xs, ys)
Z = predict(X, Y)

plt.figure(figsize=(6,6))

# plot 0.5 contour line - classify as male if above this line
plt.contour(X, Y, Z, levels=[0.5])

# classify all subjects
colors = ['lime' if i else 'yellow' for i in df.male]
ps = predict(df.weight, df.height)
errs = ((ps < 0.5) & df.male) |((ps >= 0.5) & (1-df.male))
plt.scatter(df.weight[errs], df.height[errs], facecolors='red', s=150)
plt.scatter(df.weight, df.height, facecolors=colors, edgecolors='k', s=50, alpha=1);
plt.xlabel('Weight', fontsize=16)
plt.ylabel('Height', fontsize=16)
plt.title('Gender classification by weight and height', fontsize=16)
plt.tight_layout()

### 估计一个简单继承模型的参数，Estimating parameters of a simple hierarchical model

Gelman's book has an example where the dose of a drug may be affected to the number of rat deaths in an experiment.

Gelman的书中有一个例子，说的是用药剂量可能会受到实验中大鼠死亡数量的影响。

| 剂量 Dose (log g/ml) | # 实验大鼠数目 | # 死亡大鼠数目 |
|-----------------|--------|----------|
| -0.896          | 5      | 0        |
| -0.296          | 5      | 1        |
| -0.053          | 5      | 3        |
| 0.727           | 5      | 5        |

We will model the number of deaths as a random sample from a binomial distribution, where $n$ is the number of rats and $p$ the probabbility of a rat dying. We are given $n = 5$, but we believve that $p$ may be related to the drug dose $x$. As $x$ increases the number of rats dying seems to increase, and since $p$ is a probabiliyt, we use the following model:


我们建立一个模型，把死亡数目当做一个服从二项分布（binomial distribution）的随机样本，其中 $n$ 表示实验中大鼠的数目，而 $p$ 表示单个大鼠死亡的概率。假如给定了 $n = 5$，设想我们认为死亡概率 $p$ 与用药剂量 $x$ 有关。 随着 $x$ 增长，大鼠的死亡数目也在增长，由于 $p$ 是一个概率值，所以就建立起来了下面这样的数学模型：


$$
y \sim \text{Bin}(n, p) \\
\text{logit}(p) = \alpha + \beta x \\
\alpha \sim \mathcal{N}(0, 5) \\
\beta \sim \mathcal{U}(-\infty, \infty)
$$


其中，我们将 $\alpha$ 和 $\beta$ 设置为某个模糊的先验值（vague priors），这两个是逻辑回归模型（logistic model）的参数（parameters）。



In [ ]:
# observed data
n = 5 * np.ones(4)
x = np.array([-0.896, -0.296, -0.053, 0.727])
y = np.array([0, 1, 3, 5])

with pm.Model() as model:
    # define priors
    alpha = pm.Normal('alpha', mu=0, sd=5)
    beta = pm.Flat('beta')
    
    # define likelihood
    p = invlogit(alpha + beta*x)
    y_obs = pm.Binomial('y_obs', n=n, p=p, observed=y)
    
    # inference
    start = pm.find_MAP()
    step = pm.NUTS()
    trace = pm.sample(niter, step, start, random_seed=123, progressbar=True)

In [ ]:
plt.subplot(2,2,1); 
plt.hist(trace['alpha'][-niter/2:], 25, histtype='step');
plt.subplot(2,2,2); 
plt.hist(trace['beta'][-niter/2:], 25, histtype='step');
c = plt.cm.get_cmap("jet", niter)
cvals = c(np.arange(niter))
plt.subplot(2,2,3); 
plt.plot(trace['alpha'][::10])
plt.subplot(2,2,4); 
plt.plot(trace['beta'][::10]);

### 动画演示，Animation example

参考 [Animating MCMC with PyMC3 and Matplotlib](https://github.com/twiecki/WhileMyMCMCGentlySamples/blob/master/content/downloads/notebooks/sample_animation.ipynb)


In [ ]:
from JSAnimation import IPython_display

In [ ]:
# Generate some data
np.random.seed(124)
size = 50
true_intercept = 1
true_slope = 2

x = np.linspace(0, 1, size)
y = true_intercept + x*true_slope + np.random.normal(scale=.5, size=size)

data = dict(x=x, y=y)

In [ ]:
# Quickly hacked plotting code
samples = 600
fig = plt.figure(figsize=(6, 6))
i_width = (true_intercept-.7, true_intercept+.7)
s_width = (true_slope-.7, true_slope+.7)
samples_width = (0, samples)
ax1 = fig.add_subplot(221, xlim=i_width, ylim=samples_width)
ax2 = fig.add_subplot(224, xlim=samples_width, ylim=s_width)
ax3 = fig.add_subplot(223, xlim=i_width, ylim=s_width,
                      xlabel='intercept',
                      ylabel='slope')
fig.subplots_adjust(wspace=0.0, hspace=0.0)
line1, = ax1.plot([], [], lw=1)
line2, = ax2.plot([], [], lw=1)
line3, = ax3.plot([], [], 'o', lw=2, alpha=.1)
line4, = ax3.plot([], [], lw=1, alpha=.3)
line5, = ax3.plot([], [], 'k', lw=1)
line6, = ax3.plot([], [], 'k', lw=1)
ax1.set_xticklabels([])
ax2.set_yticklabels([])
#path = plt.scatter([], [])
lines = [line1, line2, line3, line4, line5, line6]

def init():
    for line in lines:
        line.set_data([], [])
    return lines

def animate(i):
    with model:
        if i == samples * .75:
            for j in range(500): iter_sample.next() 
        trace = iter_sample.next()
    line1.set_data(trace['Intercept'][::-1], range(len(trace['Intercept'])))
    line2.set_data(range(len(trace['x'])), trace['x'][::-1])
    line3.set_data(trace['Intercept'], trace['x'])
    line4.set_data(trace['Intercept'], trace['x'])
    intercept = trace['Intercept'][-1]
    x = trace['x'][-1]
    line5.set_data([intercept, intercept], [x, s_width[1]])
    line6.set_data([intercept, i_width[1]], [x, x])
    return lines

In [ ]:
with pm.Model() as model:
    pm.glm.glm('y ~ x', data)
    step = pm.Metropolis()
    iter_sample = pm.iter_sample(samples+1000, step)

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=samples, interval=5, blit=True)

使用 PyStan
----
安装方法：

```
 pip install pystan
```

使用 Emcee
----
安装方法
```
 pip install emcee
```

In [ ]:
%load_ext version_information

%version_information pymc